# CUDA C

In [15]:
!nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2023 NVIDIA Corporation
Built on Tue_Aug_15_22:02:13_PDT_2023
Cuda compilation tools, release 12.2, V12.2.140
Build cuda_12.2.r12.2/compiler.33191640_0


In [2]:
!nvidia-smi

Fri Apr 26 19:54:30 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   58C    P8              10W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

## Multiplication Matrix

In [9]:
%%writefile mul_vectors.cu
#include <iostream>
#include <cuda.h>
#define BLOCK_SIZE 16
using namespace std;

void print_matrix(int *array, int rows, int cols) {
    for (int i = 0; i < rows; i++) {
        for (int j = 0; j < cols; j++) {
            cout << array[i * cols + j] << " ";
        }
        cout << endl;
    }
}

__global__ void mul(int *a, int *b, int*c, int c_rows, int c_cols, int common){
int row = blockIdx.y * blockDim.y + threadIdx.y;
int col = blockIdx.x  * blockDim.x + threadIdx.x;
int sum = 0;

if (col<c_cols && row<c_rows){
for (int j = 0; j<common;j++){
sum+=a[row*common + j]*b[j*c_cols + col];
}
c[c_cols*row + col] = sum;
}

}


int main(){

// initialize

const int A_r = 3;
const int A_c = 3;
const int B_r = A_c;
const int B_c = A_r;
const int C_r = A_r;
const int C_c = B_c;

const int A_size = A_r*A_c;
const int B_size  = B_r*B_c;
const int C_size = C_r*C_c;

int A[A_size] = {1, 2, 3, 4, 5, 6, 7, 8, 9};
int B[B_size] = {1, 2, 3, 4, 5, 6, 7, 8, 9};
int C[C_size];

int *m1, *m2,*result;

// allocate
cudaMallocManaged(&m1,A_size*sizeof(int));
cudaMallocManaged(&m2,B_size*sizeof(int));
cudaMallocManaged(&result,C_size*sizeof(int));

//copy data
cudaMemcpy(m1,A,A_size*sizeof(int),cudaMemcpyHostToDevice);
cudaMemcpy(m2,B,B_size*sizeof(int),cudaMemcpyHostToDevice);

// configuration
dim3 dimGrid((C_c + BLOCK_SIZE -1)/BLOCK_SIZE,(C_r+BLOCK_SIZE-1)/BLOCK_SIZE);
dim3 dimBlock(BLOCK_SIZE,BLOCK_SIZE);

mul<<<dimGrid,dimBlock>>>(m1,m2,result,C_r,C_c,A_c);
cudaDeviceSynchronize();

cudaMemcpy(C,result,C_size*sizeof(int),cudaMemcpyDeviceToHost);

cout<<"Result: \n";
print_matrix(C, C_r, C_c);


}


Overwriting mul_vectors.cu


In [10]:
!nvcc -o mul_vectors mul_vectors.cu

In [11]:
!./mul_vectors

Result: 
30 36 42 
66 81 96 
102 126 150 


## Addition Vectors

In [12]:
%%writefile add_vectors.cu
#define BLOCK_SIZE 10
# include <cuda.h>
# include <iostream>
using namespace std;

void print_array(int *arr, int size){
for (int i=0;i<size;i++){

cout<< arr[i] << " ";
}
cout<< endl;
}

__global__ void add(int *arr1, int * arr2, int * result, int size){

int block_id = blockIdx.x * blockDim.x + threadIdx.x;
if (block_id < size){
result[block_id] = arr1[block_id] + arr2[block_id];
}
}

int main(){

//initialize
int size = 10;
int arr1_cpu[size] = {1,2,3,4,5,6,7,8,9,10};
int arr2_cpu[size] = {10,9,8,7,6,5,4,3,2,1};
int result_cpu[size];

int *arr1_gpu, *arr2_gpu, *result_gpu;

//allocate memory to gpu
cudaMallocManaged(&arr1_gpu, size*sizeof(int));
cudaMallocManaged(&arr2_gpu, size*sizeof(int));
cudaMallocManaged(&result_gpu,size*sizeof(int));

//copy data from cpu to gpu
cudaMemcpy(arr1_gpu,arr1_cpu,size*sizeof(int),cudaMemcpyHostToDevice);
cudaMemcpy(arr2_gpu,arr2_cpu,size*sizeof(int),cudaMemcpyHostToDevice);

// Configuration for Execution
dim3 dimGrid((size+BLOCK_SIZE-1)/BLOCK_SIZE);
dim3 dimBlock(BLOCK_SIZE);

//Launch Kernel
add<<<dimGrid,dimBlock>>>(arr1_gpu,arr2_gpu,result_gpu,size);
cudaDeviceSynchronize();

// copy data from gpu to cpu
cudaMemcpy(result_cpu,result_gpu,size*sizeof(int),cudaMemcpyDeviceToHost);

cout<<"GPU Result: \n";
print_array(result_cpu,size);


}


Writing add_vectors.cu


In [13]:
!nvcc -o add_vectors add_vectors.cu

In [14]:
!./add_vectors

GPU Result: 
11 11 11 11 11 11 11 11 11 11 
